<a href="https://colab.research.google.com/github/monu322/CE888/blob/main/Assg2/OffenseDetection/Topic_number_for_Offensive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis

In [3]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
#importing offesive dataset

tweets_offensive=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt', header=None)
print('Training tweets', tweets_offensive.shape)

tweets_offensive_labels=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt', header=None)
print('Training labels', tweets_offensive_labels.shape)

tweets_offensive_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt', header=None)
print('Test tweets',tweets_offensive_test.shape)

tweets_offensive_test_labels=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt', header=None)
print('Test labels',tweets_offensive_test_labels.shape)



Training tweets (11916, 51)
Training labels (11916, 1)
Test tweets (860, 3)
Test labels (860, 1)


In [34]:
tweets_offensive = tweets_offensive[[0]]
tweets_offensive.head()

,0
0,@user Bono... who cares. Soon people will unde...
1,@user Eight years the republicans denied obama...
2,@user Get him some line help. He is gonna be j...
3,@user @user She is great. Hi Fiona!
4,@user She has become a parody unto herself? Sh...


In [37]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def strip_newline(series):
    return [review.replace('\n','') for review in series]

tweets_offensive[0] = strip_newline(tweets_offensive[0])
tweets_offensive[0][20:42].values

array(['@user You are so fragile. It must be hard to live that way.',
       '@user All you do is lie. All congress does us lie.  MAGA.',
       '@user Freudian slip.',
       '@user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user',
       "@user @user Why are you wasting time replying on twitter (trolling) on behalf of people making under that amount?  Seems like someone is just a propaganda pusher...or likes gloating to other #MAGA's that they got screwed by Trump.",
       '@user We had Leveon Bell. Thats the dumbest argument. But we can compare college for when he played. His best season was 1700 yds 26 TDs. How about Collins? And this is Conners first year starting and he is starting much better than Collins',
       '@user @user SHE IS A FUCKING MESS!! I HATE HE

In [38]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

tweet_words = list(sent_to_words(tweets_offensive[0]))

tweet_words[21][:10]

['user', 'all', 'you', 'do', 'is', 'lie', 'all', 'congress', 'does', 'us']

In [39]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

tweet_words = remove_stopwords(tweet_words)

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(tweet_words)

print(trigram_tr[bigram_tr[tweet_words[0]]][:200])

['user', 'bono', 'cares', 'soon', 'people', 'understand', 'gain', 'nothing', 'following', 'phony', 'celebrity', 'become', 'leader', 'people', 'instead', 'help', 'support', 'fellow', 'countrymen']


In [40]:
nlp = spacy.load('en', disable=['parser', 'ner'])

warnings.filterwarnings("ignore")

def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

trigrams_tr = [trigram_tr[bigram_tr[review]] for review in tweet_words]

lemma_lg = lemmatization(trigrams_tr)

In [43]:
lemma_lg[0][:20]

id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

corpus_lg[22][:20]

[]

In [44]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)

len(hdp.print_topics())

20

In [45]:
hdp.print_topics(num_topics=20)

[(0,
  '0.006*opposite + 0.005*rapist + 0.005*somehow + 0.004*almost + 0.004*regard + 0.004*finally + 0.004*holder + 0.004*rather + 0.004*traitor + 0.004*mad'),
 (1,
  '0.006*course + 0.005*reason + 0.005*excuse + 0.005*ill + 0.005*picture + 0.004*boy + 0.004*liberal + 0.004*stand + 0.004*insane + 0.004*forever'),
 (2,
  '0.006*completely + 0.005*growth + 0.005*hunt + 0.005*bigot + 0.005*regard + 0.004*later + 0.004*remind + 0.004*attempt + 0.004*last + 0.004*brown'),
 (3,
  '0.007*woman + 0.006*pm + 0.005*dead + 0.005*promise + 0.004*stock + 0.004*economy + 0.004*yesterday + 0.004*screw + 0.004*decide + 0.004*invite'),
 (4,
  '0.006*socialist + 0.005*year_old + 0.005*wake + 0.004*clean + 0.004*nee + 0.004*instead + 0.004*mother + 0.004*protection + 0.004*fair + 0.004*profit'),
 (5,
  '0.006*usual + 0.005*weapon + 0.004*one + 0.004*could + 0.004*choice + 0.004*recognize + 0.004*exist + 0.004*connect + 0.004*gun_control_laws + 0.004*however'),
 (6,
  '0.005*measure + 0.005*fake + 0.004*